<a href="https://colab.research.google.com/github/Nataliawijayaa/Credit_Scoring_Analysis/blob/main/Credit_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Borrower Default Risk

My task is to prepare a report for the credit division of a bank. I will investigate the influence of a borrower's marital status and the number of children they have on the probability of defaulting on loan repayment. The bank already has some data on the creditworthiness of borrowers.

My report will be taken into consideration when conducting **credit assessments** for prospective customers. **Credit assessments** are used to evaluate the ability of potential borrowers to repay their loans.

## Open the data *file* and read its general information


In [3]:
# Load all *libraries*
import pandas as pd

# Load the data
df = pd.read_csv('/content/credit_scoring_eng.csv')

## Chapter 1. Data Exploration

**Data Description**
- `children` - number of children in the family
- `days_employed` - customer's work experience in days
- `dob_years` - customer's age in years
- `education` - customer's education level
- `education_id` - identifier for customer's education level
- `family_status` - identifier for customer's marital status
- `family_status_id` - identifier for marital status
- `gender` - customer's gender
- `income_type` - type of employment
- `debt` - whether the customer has ever defaulted on a loan
- `total_income` - monthly income
- `purpose` - purpose of obtaining a loan

In [4]:
# number of rows and columns in dataset
df.shape


(21525, 12)

In [5]:
# display the first 10 rows
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Based on the first 10 data, there are some anomalies in the `days_employed` column, where it is unusual to have negative values. As for the `education` data, there are inconsistencies in the capitalization, where some entries are capitalized and others are not. However, the other data in the first 10 entries appear to be fine.

In [6]:
# general information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Based on the general information, there are missing values in two columns, namely `days_employed` and `total_income`.

In [7]:
# Let's take a look at the filtered table with missing values 
df[df['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Based on the analysis so far, the dataset contains missing values in two columns: `days_employed` and `total_income`. The missing values appear to be symmetric, as observed in the table above, which shows that the missing values in `days_employed` also have missing values in `total_income`.


In [8]:
# Let's apply some conditions to filter the data and see the number of rows in the filtered table.
df[df['days_employed'].isna() & df['total_income'].isna()].shape

(2174, 12)

**Preliminary Conclusion**

The number of rows in the filtered table matches the number of missing values, indicating that the missing values in the table are indeed symmetric. Symmetric here means that each row with a missing value in the `days_employed` column will also have a missing value in the `total_income` column.


In [9]:
# Let's examine the customers who do not have data on identified characteristics and columns with missing values.
df_null = df[df['days_employed'].isna() & df['total_income'].isna()]
df_null


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [10]:
# Check the distribution.
# Distribution of employment types.
a = df_null['income_type'].value_counts(normalize=True).reset_index().rename(columns={'income_type':'percentage'})
a['percentage'] = a['percentage'].apply('{:,.2%}'.format)
b = df_null['income_type'].value_counts().reset_index().rename(columns={'income_type':'count'})
c = pd.concat([a, b[['count']]], axis = 1)
c

,index,percentage,count
0,employee,50.83%,1105
1,business,23.37%,508
2,retiree,19.00%,413
3,civil servant,6.76%,147
4,entrepreneur,0.05%,1


Based on the distribution table of employment types, the missing values in `days_employed` and `total_income` are not specific to any employment type, as half of the missing values are from employees.

**Possible reasons for missing values in the data**

Let's examine the causes behind the missing values, whether they occur randomly or exhibit any distinct patterns.

In [11]:
# Examining the distribution across the entire *dataset* 
# Examining the distribution of occupation types across the entire *dataset*
d = df['income_type'].value_counts(normalize=True).reset_index().rename(columns={'income_type':'percentage'})
d['percentage'] = d['percentage'].apply('{:,.2%}'.format)
e = df['income_type'].value_counts().reset_index().rename(columns={'income_type':'count'})
f = pd.concat([d, e[['count']]], axis = 1)
f


,index,percentage,count
0,employee,51.66%,11119
1,business,23.62%,5085
2,retiree,17.91%,3856
3,civil servant,6.78%,1459
4,unemployed,0.01%,2
5,entrepreneur,0.01%,2
6,student,0.00%,1
7,paternity / maternity leave,0.00%,1


**Preliminary Conclusion**

The distribution in the original dataset is nearly similar to the distribution in the filtered table, where the percentage of missing values in each job type index is almost the same as the percentage in the original dataset.

However, we cannot draw any conclusions yet. Let's further investigate the *dataset*.


In [12]:
# Examine the causes and other patterns that could result in missing values
# Check the educational background
df_null['education'].value_counts(normalize=True)

secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
Some College           0.003220
SOME COLLEGE           0.003220
Primary Education      0.000460
PRIMARY EDUCATION      0.000460
Name: education, dtype: float64

In [13]:
df['education'].value_counts(normalize=True)

secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
Graduate Degree        0.000046
GRADUATE DEGREE        0.000046
Name: education, dtype: float64

**Preliminary Conclusion**

It appears that the missing values are random in nature, as there are no discernible patterns observed based on the occupation or educational background.

In [14]:
# Examine other patterns 
# Check the relationship between income_type and age using a pivot table
df_null_pivot = df_null.pivot_table(index='dob_years', columns='income_type', values='debt', aggfunc='count', margins=True)
df_null_pivot

income_type,business,civil servant,employee,entrepreneur,retiree,All
dob_years,,,,,,
0,2.0,NaN,5.0,NaN,3.0,10
19,NaN,NaN,1.0,NaN,NaN,1
20,1.0,NaN,4.0,NaN,NaN,5
21,7.0,1.0,10.0,NaN,NaN,18
22,6.0,NaN,11.0,NaN,NaN,17
23,5.0,1.0,30.0,NaN,NaN,36
24,9.0,1.0,10.0,NaN,1.0,21
25,4.0,4.0,15.0,NaN,NaN,23
26,9.0,2.0,24.0,NaN,NaN,35


**Conclusion**

I did not find any specific patterns, indicating that the missing values in the `days_employed` and `total_income` columns are random or coincidental. This conclusion is supported by various checks, including analysis of employment type, educational background, and age, which did not reveal any distinct patterns. Therefore, it can be inferred that the missing values can occur across different employment types (employee, civil servant, business) and age ranges.

## Data Transformation

In [15]:
# Let's examine all the values in the education column to check for any spelling errors that need to be corrected
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [16]:
# Correct any misspellings
df['education'] = df['education'].str.lower()

In [17]:
# Check all values in the column to ensure that we have corrected them accurately
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [18]:
# Let's examine the distribution values in `children` 
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In the `children` column, there are values that are unusual and not logically possible, such as -1 indicating a negative number of children or 20 indicating an extremely high number of children. These values could be the result of data entry errors or typos, where it is possible that -1 was intended to be 1 and 20 could be a typo for 2. However, since the proportion of those unusual data is quite small, and I cannot accurately determine what happened with those data points, thus I have decided to remove them from the dataset.


In [19]:
# Removing the data points with values 20 and -1 in the `children` column
df = df.drop(df[df['children'] == -1].index)
df = df.drop(df[df['children'] == 20].index)

In [20]:
# Recheck the `children` column to ensure that all the values have been removed
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [21]:
# Find any problematic data in the `days_employed` column
df['days_employed'].describe()

count     19240.000000
mean      63159.820777
std      140928.943329
min      -18388.949901
25%       -2747.235601
50%       -1203.934202
75%        -289.740178
max      401755.400475
Name: days_employed, dtype: float64

In [22]:
df[df['days_employed'] < 0].shape

(15809, 12)

Initially, it appears that there are data in the `days_employed` column with negative values. This already indicates unusual values.

The number of problematic data can be considered high, as it accounts for more than half of the total data. Therefore, we cannot simply remove these problematic data. The negative values in the `days_employed` column may be due to sign errors in the data. Thus, since the `days_employed` data cannot be negative, I will convert all of them to positive values. Additionally, there are other peculiar values, such as customers with a work experience of 900+ years.

In [23]:
# Fix the problematic values
df['days_employed'] = df['days_employed'].abs()


In [24]:
# Check the results - make sure that the issues have been resolved.
df['days_employed'].describe()

count     19240.000000
mean      67027.691459
std      139130.846446
min          24.141633
25%         927.984311
50%        2195.251592
75%        5556.372075
max      401755.400475
Name: days_employed, dtype: float64

In [25]:
df['year_employed'] = df['days_employed'] / 365

In [26]:
df.loc[df['year_employed']>50].min()

children                              0
days_employed              18388.949901
dob_years                             0
education             bachelor's degree
education_id                          0
family_status         civil partnership
family_status_id                      0
gender                                F
income_type                    employee
debt                                  0
total_income                   3306.762
purpose             building a property
year_employed                 50.380685
dtype: object

In [27]:
# Finding the minimum value of `year_employed`
df.loc[(df['year_employed']>50) & (df['income_type']=='retiree')].min()

children                              0
days_employed             328728.720605
dob_years                             0
education             bachelor's degree
education_id                          0
family_status         civil partnership
family_status_id                      0
gender                                F
income_type                     retiree
debt                                  0
total_income                   3306.762
purpose             building a property
year_employed                900.626632
dtype: object

In [28]:
# Changing the values of `days_employed` that are greater than 18250 days (50 years) to 18250 days
df.loc[df['days_employed'] > 18250, 'days_employed'] = 18250

In [29]:
df['year_employed'] = df['days_employed'] / 365

In [30]:
# Checking the values
df['days_employed'].describe()

count    19240.000000
mean      5188.384980
std       6433.648220
min         24.141633
25%        927.984311
50%       2195.251592
75%       5556.372075
max      18250.000000
Name: days_employed, dtype: float64

In [31]:
df['year_employed'].describe()

count    19240.000000
mean        14.214753
std         17.626433
min          0.066141
25%          2.542423
50%          6.014388
75%         15.222937
max         50.000000
Name: year_employed, dtype: float64

In [32]:
# Check `dob_years` for any suspicious values
df['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
22    183
66    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [33]:
df['dob_years'].value_counts(normalize=True)

35    0.028689
40    0.028175
41    0.028175
34    0.027895
38    0.027801
42    0.027661
33    0.026960
39    0.026726
31    0.025979
36    0.025839
44    0.025371
29    0.025371
30    0.025044
48    0.025044
37    0.024811
43    0.023830
50    0.023783
32    0.023643
49    0.023596
28    0.023409
45    0.023082
27    0.022895
52    0.022568
56    0.022521
47    0.022428
54    0.022241
46    0.021914
58    0.021540
57    0.021353
53    0.021353
51    0.020839
59    0.020606
55    0.020606
26    0.018970
60    0.017568
25    0.016634
61    0.016494
62    0.016400
63    0.012522
64    0.012289
24    0.012289
23    0.011775
65    0.009065
22    0.008551
66    0.008551
67    0.007803
21    0.005140
0     0.004672
68    0.004626
69    0.003878
70    0.003037
71    0.002710
20    0.002383
72    0.001542
19    0.000654
73    0.000374
74    0.000280
75    0.000047
Name: dob_years, dtype: float64

There is an issue with the customer's age, where there are values of 0 in the `dob_years` column. Since the percentage of customers with age 0 is quite small, which is below 0.005, we can remove those problematic values.

In [34]:
# Handle the issue in the `dob_years` column
df = df.drop(df[df['dob_years'] == 0]. index)

In [35]:
# Check the result
df['dob_years'].value_counts()

35    614
41    603
40    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [36]:
# Examine the values for the `family_status` column.
df['family_status'].value_counts()


married              12254
civil partnership     4139
unmarried             2783
divorced              1179
widow / widower        947
Name: family_status, dtype: int64

There are no issues found in the `family_status` column.

In [37]:
# Examine the values in the `gender` column.
df['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

In [38]:
# Fix any problematic values
df = df.drop(df[df['gender']=='XNA'].index)

In [39]:
# Check the results
df['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

In [40]:
# Examine the values in the `income_type` column
df['income_type'].value_counts()

employee                       10996
business                        5033
retiree                         3819
civil servant                   1447
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

There are no issues found in the `income_type` column.

In [41]:
# Check for duplicates
df.duplicated().sum()
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,year_employed
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,NaN
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,NaN
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,NaN
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education,NaN
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated,NaN
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation,NaN
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate,NaN


In [42]:
# Resolve duplicates
df = df.drop_duplicates().reset_index(drop=True)

In [43]:
# Recheck duplicates
df.duplicated().sum()

0

In [44]:
# Check the current size of the dataset after after manipulating the data
df.shape

(21230, 13)

There have been several changes in the new dataset, where there are no more duplicate data, and the problematic or unusual data have been resolved.

# Working with Missing Values

In [45]:
# education dictionary
education_dic = df[['education', 'education_id']]
education_dic.value_counts()

education            education_id
secondary education  1               15012
bachelor's degree    0                5192
some college         2                 738
primary education    3                 282
graduate degree      4                   6
dtype: int64

In [46]:
education_test = dict(zip(education_dic['education'], education_dic['education_id']))
education_test

{"bachelor's degree": 0,
 'secondary education': 1,
 'some college': 2,
 'primary education': 3,
 'graduate degree': 4}

In [47]:
# family_status dictionary
family_status_dic = df[['family_status', 'family_status_id']]
family_status_dic.value_counts()

family_status      family_status_id
married            0                   12213
civil partnership  1                    4112
unmarried          4                    2780
divorced           3                    1179
widow / widower    2                     946
dtype: int64

In [48]:
family_status_test = dict(zip(family_status_dic['family_status'], family_status_dic['family_status_id']))
family_status_test

{'married': 0,
 'civil partnership': 1,
 'widow / widower': 2,
 'divorced': 3,
 'unmarried': 4}

### Fixing Missing Values in `total_income`

Handling missing values in `total_income` and `days_employed` can be done by filling the missing values with either the median or the mean value.


In [49]:
# a function to calculate age categories
# age groups <10, 11-20, 21-30, 31-40, 41-50, 51-60, >70.

def age_group(age): 
    try:
        if age <= 10:
            return '< 10'
        if age <= 20:
            return '11-20'
        if age <= 30:
            return '21-30'
        if age <= 40:
            return '31-40'
        if age <= 50:
            return '41-50'
        if age <= 60:
            return '51-60'
        if age <= 70:
            return '61-70'
        else:
            return '> 70'
    except:
        return 0

In [50]:
# testing the function
age_group(27)

'21-30'

In [51]:
# Create a new column based on the function.
df['age_group'] = df['dob_years'].apply(age_group)


In [52]:
# Check the values
df


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,year_employed,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,41-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,31-40
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,31-40
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,31-40
4,0,18250.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50.000000,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21225,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087,41-50
21226,0,18250.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,50.000000,61-70
21227,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991,31-40
21228,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347,31-40


One of the factors that can affect income is the age of the customer.

In [53]:
# Create a table without missing values
df_non_nan = df[df['days_employed'].notnull()]
df_non_nan.shape

(19149, 14)

In [54]:
# Observe the average income values based on the age_group
mean_total_income = df_non_nan.pivot_table(index='age_group', values='total_income', aggfunc='mean')
mean_total_income

,total_income
age_group,
11-20,19586.303559
21-30,25934.233444
31-40,28391.459775
41-50,28402.209709
51-60,25479.278587
61-70,23238.686225
> 70,19575.454327


In [55]:
# Observe the median income values based on the age_group
median_total_income = df_non_nan.pivot_table(index='age_group', values='total_income', aggfunc='median')
median_total_income

,total_income
age_group,
11-20,17257.2770
21-30,23079.3820
31-40,24850.0920
41-50,24572.2550
51-60,22050.9965
61-70,19700.2560
> 70,18611.5935


In [56]:
# The median values based on `age_group` and `education` 
df_non_nan.pivot_table(index='age_group', columns='education', values='total_income', aggfunc='median')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
age_group,,,,,
11-20,14251.5510,NaN,NaN,20346.5530,13908.3910
21-30,26351.1500,NaN,23388.8070,21364.5815,23103.8030
31-40,28969.7255,18187.3015,19674.2825,23080.7780,28829.7110
41-50,30479.7070,31771.3210,21807.6680,22809.6040,29497.7090
51-60,27665.2370,42945.7940,18022.0315,21016.0275,22718.9595
61-70,25193.1730,28334.2150,16240.8440,18794.6800,27938.4350
> 70,26223.0685,NaN,15013.5050,18146.7015,19946.7950


In [57]:
# The median values based on `age_group` dan `income_type`
df_non_nan.pivot_table(index='age_group', columns='income_type', values='total_income', aggfunc='median')

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed
age_group,,,,,,,,
11-20,19648.2745,12125.9860,16356.939,NaN,NaN,NaN,NaN,NaN
21-30,25865.3170,23300.0970,21916.289,79866.103,NaN,13553.0235,15712.26,NaN
31-40,28955.2600,24965.7220,23304.690,NaN,8612.661,18475.1010,NaN,9593.119
41-50,28395.9820,23847.2850,22952.040,NaN,NaN,23079.7810,NaN,32435.602
51-60,27390.1930,23392.7575,22686.171,NaN,NaN,19251.0445,NaN,NaN
61-70,29171.9890,25144.1470,23521.586,NaN,NaN,18414.6660,NaN,NaN
> 70,28138.8950,19530.6775,20246.123,NaN,NaN,18146.7015,NaN,NaN


According to my analysis, I believe that the most influential factors for income are `age_group`. Therefore, I suggest replacing the missing values with the median value for each respective age group. Since there are significant differences in income values across age groups, it is more appropriate to use the median as a replacement for missing values rather than the mean. This approach will help ensure that the imputed values align better with the existing income distribution within each age group.

In [58]:
# Write a function that will be used to fill missing values
# Fill missing values with the median total_income based on age_group

def get_median_income(age_group):
    try:
        return median_total_income['total_income'][age_group]
    except:
        return 'error'      

In [59]:
# Check the function
get_median_income('21-30')

23079.382

In [60]:
# Apply the function to each row
# Create a new column containing the median value in total_income.
df['median_income'] = df['age_group'].apply(get_median_income)

In [61]:
# Check if there are any errors
df[df['median_income']=='error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,year_employed,age_group,median_income


In [62]:
# Filling missing values with the median
df['total_income'] = df['total_income'].fillna(df['median_income'])

In [63]:
# Check the number of entries in the column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  year_employed     19149 non-null  float64
 13  age_group         21230 non-null  object 
 14  median_income     21230 non-null  float64
dtypes: float64(4), int64(5), object(6)
memory usage: 2.4+ MB


###  Fixing values in `days_employed`

In [64]:
# Distribution of median values of days_employed based on the identified parameter
# Assume that `days_employed` is influenced by `income_type`
median_days_employed = df_non_nan.pivot_table(index='income_type', values='days_employed', aggfunc='median')
median_days_employed


,days_employed
income_type,
business,1555.993659
civil servant,2672.903939
employee,1573.791064
entrepreneur,520.848083
paternity / maternity leave,3296.759962
retiree,18250.000000
student,578.751554
unemployed,18250.000000


In [65]:
# Distribution of mean values of `days_employed` based on `income_type`
mean_days_employed = df_non_nan.pivot_table(index='income_type', values='days_employed', aggfunc='mean')
mean_days_employed

,days_employed
income_type,
business,2119.512866
civil servant,3392.119263
employee,2325.726856
entrepreneur,520.848083
paternity / maternity leave,3296.759962
retiree,18250.000000
student,578.751554
unemployed,18250.000000


In [66]:
df_non_nan.pivot_table(index='age_group', columns='income_type', values='days_employed', aggfunc='mean')

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed
age_group,,,,,,,,
11-20,515.985625,600.437357,782.125220,NaN,NaN,NaN,NaN,NaN
21-30,1221.631984,1700.554589,1271.929210,520.848083,NaN,18250.0,578.751554,NaN
31-40,1905.265469,2950.167522,2067.728443,NaN,3296.759962,18250.0,NaN,18250.0
41-50,2516.604108,4046.640714,2774.830948,NaN,NaN,18250.0,NaN,18250.0
51-60,2915.078089,4944.394327,3269.414788,NaN,NaN,18250.0,NaN,NaN
61-70,3644.075107,4390.716537,3861.945117,NaN,NaN,18250.0,NaN,NaN
> 70,4622.013775,1204.922098,4266.205758,NaN,NaN,18250.0,NaN,NaN


I will use the median value to replace the missing values in `days_employed` because there is a significant difference in the values. Therefore, it is better to use the median value instead of the mean value.


In [67]:
# Function to calculate the median based on income_type

def get_median_days_employed(income_type):
    try:
        return median_days_employed['days_employed'][income_type]
    except:
        return 'error'

In [68]:
# Check the function
get_median_days_employed('retiree')

18250.0

In [69]:
# Apply the function to income_type
df['median_days'] = df['income_type'].apply(get_median_days_employed)

In [70]:
# check the data
df


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,year_employed,age_group,median_income,median_days
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,41-50,24572.2550,1573.791064
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,31-40,24850.0920,1573.791064
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,31-40,24850.0920,1573.791064
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,31-40,24850.0920,1573.791064
4,0,18250.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50.000000,51-60,22050.9965,18250.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21225,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087,41-50,24572.2550,1555.993659
21226,0,18250.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,50.000000,61-70,19700.2560,18250.000000
21227,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991,31-40,24850.0920,1573.791064
21228,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347,31-40,24850.0920,1573.791064


In [71]:
# Fill the missing values
df['days_employed'] = df['days_employed'].fillna(df['median_days'])

In [72]:
# Check the entries in all columns - make sure we have fixed all missing values
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
year_employed       2081
age_group              0
median_income          0
median_days            0
dtype: int64

## Data Categorization

In [73]:
# Display the values of the data for categorization
df['purpose']


0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21225       housing transactions
21226          purchase of a car
21227                   property
21228          buying my own car
21229               to buy a car
Name: purpose, Length: 21230, dtype: object

In [74]:
# Check unique values
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Based on the data above, it can be identified that the `purpose` column can be categorized into 4 categories, namely: housing, car, education, and wedding.


In [75]:
# Function to categorize the data
df['purpose_category'] = df['purpose']

def replace_wrong_values(wrong_values, correct_values):
    for wrong_value in wrong_values:
        df['purpose_category'] = df['purpose_category'].replace(wrong_values, correct_values)
    
wrong_housing = ['purchase of the house', 'housing transactions', 
         'purchase of the house for my family', 'buy real estate', 
         'buy commercial real estate', 'buy residential real estate', 
         'construction of own property', 'property', 'building a property',
         'transactions with commercial real estate', 'building a real estate',
         'transactions with my real estate', 'purchase of my own house', 'real estate transactions',
         'buying property for renting out', 'housing renovation']
correct_housing = 'housing'

replace_wrong_values(wrong_housing, correct_housing)


wrong_car = ['car purchase', 'buying a second-hand car', 'buying my own car', 'cars', 'second-hand car purchase',
            'to own a car', 'purchase of a car', 'to buy a car']
correct_car = 'car'

replace_wrong_values(wrong_car, correct_car)


wrong_education = ['supplementary education', 'education', 'to become educated', 'getting an education', 
                   'to get a supplementary education', 'getting higher education','profile education', 
                   'university education', 'going to university']
correct_education = 'education'

replace_wrong_values(wrong_education, correct_education)


wrong_wedding = ['to have a wedding', 'having a wedding', 'wedding ceremony']
correct_wedding = 'wedding'

replace_wrong_values(wrong_wedding, correct_wedding)

In [76]:
# Create a column contains categories and calculate their values
df['purpose_category'].value_counts()

housing      10703
car           4258
education     3970
wedding       2299
Name: purpose_category, dtype: int64

In [77]:
# Numeric data in the column for categorization
df['total_income']


0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21225    35966.698
21226    24959.969
21227    14347.610
21228    39054.888
21229    13127.587
Name: total_income, Length: 21230, dtype: float64

In [78]:
# Statistical summary for the column
df['total_income'].describe()


count     21230.000000
mean      26458.605096
std       15727.848280
min        3306.762000
25%       17208.100500
50%       23228.204000
75%       31327.351000
max      362496.645000
Name: total_income, dtype: float64

Categorization in the `total_income` column:
* If income is below 10000, it will be categorized as low income
* If income is below 30000 and above 10000, it will be categorized as middle income
* If income is above 30000, it will be categorized as high income

In [79]:
# Create a function to categorize total_income into income groups
def total_income_group(total_income):
    try:
        if 0 < total_income <= 10000:
            return 'low'
        if 10000 < total_income <= 30000:
            return 'middle'
        else:
            return 'high'
    except:
        return 0


In [80]:
# Create a column that contains the categories
df['income_group'] = df['total_income'].apply(total_income_group)

In [81]:
# Calculate each category value to see its distribution.
df['income_group'].value_counts()

middle    14455
high       5859
low         916
Name: income_group, dtype: int64

## Hypothesis Testing


**Is there any correlation between having children and the probability of loan default?**

In [82]:
# Check the data for children and debt
pivot_table_children = df.pivot_table(index='children', columns = 'debt', values='dob_years', aggfunc='count')

# Calculate the percentage of loan defaults based on the number of children
pivot_table_children['percentage'] = pivot_table_children[1]/(pivot_table_children[0]+pivot_table_children[1])*100
pivot_table_children


debt,0,1,percentage
children,,,
0,12963.0,1058.0,7.545824
1,4351.0,441.0,9.202838
2,1845.0,194.0,9.514468
3,301.0,27.0,8.231707
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Conclusion**

Based on my analysis from the data above, it can be concluded that there is a correlation between having children and the probability of loan default, as individuals with children have a higher percentage of loan defaults compared to those without children. However, this does not apply to individuals with 5 children, as none of them have defaulted and they have fully repaid their loans. This may be due to the very low frequency of individuals with 5 children. Consequently, the bank may consider granting loans to individuals who do not have children, while for those with children, additional factors should be taken into account, such as their income type or the purpose of getting a loan. 

**Is there any correlation between marital status and the probability of loan default?**

In [83]:
# Check the data for marital status and debt
pivot_table_status = df.pivot_table(index='family_status', columns='debt', values='age_group', aggfunc='count')

# Calculate the percentage of loan defaults based on family status
pivot_table_status['percentage'] =  pivot_table_status[1]/(pivot_table_status[0]+pivot_table_status[1])*100
pivot_table_status


debt,0,1,percentage
family_status,,,
civil partnership,3729,383,9.314202
divorced,1095,84,7.124682
married,11290,923,7.557521
unmarried,2508,272,9.784173
widow / widower,884,62,6.553911


**Conclusion**

Based on the data above, individuals who are unmarried have the highest percentage of loan defaults compared to others. However, in my opinion, there is no correlation between marital status and the probability of loan default. This is because if it is assumed that unmarried individuals are less likely to repay the loan, it does not explain why divorced individuals are able to repay their loans, considering they have no dependents. Hence, it has no correlation based on my analysis.


**Is there any correlation between income level and the probability of loan default?**

In [84]:
# Check the data for income level and debt
pivot_table_income = df.pivot_table(index='income_group', columns='debt', values='age_group', aggfunc='count')


# Calculate the percentage of loan defaults based on income level
pivot_table_income['percentage'] =  pivot_table_income[1]/(pivot_table_income[0]+pivot_table_income[1])*100
pivot_table_income


debt,0,1,percentage
income_group,,,
high,5426,433,7.390340
low,858,58,6.331878
middle,13222,1233,8.529920


**Conclusion**

Based on my analysis, there is a correlation between income level and the probability of loan default, where the middle income group has the highest percentage of loan defaults compared to others. However, oddly enough, those with low income actually have the lowest percentage of loan defaults, whereas high-income individuals should ideally have the lowest default rate.

There could be various factors contributing to this observation. It is possible that individuals with middle income face financial challenges, such as higher expenses or debt burdens, that make it more difficult for them to meet loan repayment obligations. On the other hand, individuals with low income might have a stronger sense of financial responsibility and are more cautious when it comes to borrowing, leading to a lower default rate despite their limited income. Therefore, further analysis and investigation are needed to fully understand the underlying reasons behind these findings.

**How does the purpose of the loan affect the percentage of loan defaults?**

In [85]:
# Check the percentage of loan default for each loan purpose and perform analysis.
pivot_table_purpose = df.pivot_table(index='purpose_category', columns='debt', values='age_group', aggfunc='count')
pivot_table_purpose['percentage'] =  pivot_table_purpose[1]/(pivot_table_purpose[0]+pivot_table_purpose[1])*100
pivot_table_purpose

debt,0,1,percentage
purpose_category,,,
car,3861,397,9.323626
education,3601,369,9.294710
housing,9926,777,7.259647
wedding,2118,181,7.872988


**Conclusion**

Based on the data above, the highest percentage of loan defaults is observed among those who want to buy a car, while the lowest is among those who buy a house. The higher default rate for car purchases may be attributed to customers' tendency to make impulsive purchases for daily convenience or luxury items. On the other hand, housing is a fundamental and essential need, making loan repayment for housing purposes a top priority.

# Conclusion

Pre-processing:
1. The missing values were found to be random occurrences or system errors, and they were filled with the median values.
1. Duplicate values were identified and removed from the dataset.
1. Anomalies in the columns have been transformed.


Conclusions regarding the questions asked:
1. Customers with children have a higher probability of loan default compared to those without children.
1. Unmarried customers have the highest probability of loan default compared to other marital statuses.
1. Customers with middle-income levels have a higher probability of loan default compared to other income levels.
1. Customers who take loans for the purpose of purchasing a car have a higher probability of loan default compared to other purposes.